$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
$$
# Part 4: Convolutional Neural Networks
<a id=part4></a>

In this part we will explore convolution networks. We'll implement a common block-based deep CNN pattern with an without residual connections.

In [6]:
import os
import re
import sys
import glob
import numpy as np
import matplotlib.pyplot as plt
import unittest
import torch
import torchvision
import torchvision.transforms as tvtf

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [7]:
seed = 42
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
plt.rcParams.update({'font.size': 12})
test = unittest.TestCase()

In [4]:
print("CUDA Available:", torch.cuda.is_available())

# If CUDA is available, print additional details
if torch.cuda.is_available():
    print("Number of GPUs:", torch.cuda.device_count())
    print("Current Device:", torch.cuda.current_device())
    print("Device Name:", torch.cuda.get_device_name(torch.cuda.current_device()))

CUDA Available: False


In [ ]:
#I have an Apple Mac Mini M2 which uses Metal Performance Shaders (MPS) for GPU
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Use Metal (MPS) on Apple Silicon
else:
    device = torch.device("cpu")  # Fall back to CPU
print("Using device:", device)

Using device: mps


## Reminder: Convolutional layers and networks
<a id=part3_1></a>

Convolutional layers are the most essential building blocks of the state of the art deep learning image classification models and also play an important role in many other tasks.
As we saw in the tutorial, when applied to images, convolutional layers operate on and produce volumes (3D tensors) of activations.

A convenient way to interpret convolutional layers for images is as a collection of 3D learnable filters,
each of which operates on a small spatial region of the input volume.
Each filter is convolved with the input volume ("slides over it"),
and a dot product is computed at each location followed by a non-linearity which produces one activation.
All these activations produce a 2D plane known as a **feature map**.
Multiple feature maps (one for each filter) comprise the output volume.

<img src="imgs/cnn_filters.png" width="600" />


A crucial property of convolutional layers is their translation equivariance, i.e. shifting the input results in
and equivalently shifted output.
This produces the ability to detect features regardless of their spatial location in the input.

Convolutional network architectures usually follow a pattern basic repeating blocks: one or more convolution layers, each followed by a non-linearity (generally ReLU) and then a pooling layer to reduce spatial dimensions. Usually, the number of convolutional filters increases the deeper they are in the network.
These layers are meant to extract features from the input.
Then, one or more fully-connected layers is used to combine the extracted features into the required number of output class scores.

## Building convolutional networks with PyTorch
<a id=part3_2></a>

PyTorch provides all the basic building blocks needed for creating a convolutional arcitecture within the [`torch.nn`](https://pytorch.org/docs/stable/nn.html) package.
Let's use them to create a basic convolutional network with the following architecture pattern:

    [(CONV -> ACT)*P -> POOL]*(N/P) -> (FC -> ACT)*M -> FC

Here $N$ is the total number of convolutional layers,
$P$ specifies how many convolutions to perform before each pooling layer
and $M$ specifies the number of hidden fully-connected layers before the final output layer.

**TODO**: Complete the implementaion of the `CNN` class in the `hw3/cnn.py` module.
Use PyTorch's `nn.Conv2d` and `nn.MaxPool2d` for the convolution and pooling layers.
It's recommended to implement the missing functionality in the order of the class' methods.

In [8]:
from hw3.cnn import CNN

test_params = [
    dict(
        in_size=(3,100,100), out_classes=10,
        channels=[32]*4, pool_every=2, hidden_dims=[100]*2,
        conv_params=dict(kernel_size=3, stride=1, padding=1),
        activation_type='relu', activation_params=dict(),
        pooling_type='max', pooling_params=dict(kernel_size=2),
    ),
    dict(
        in_size=(3,100,100), out_classes=10,
        channels=[32]*4, pool_every=2, hidden_dims=[100]*2,
        conv_params=dict(kernel_size=5, stride=2, padding=3),
        activation_type='lrelu', activation_params=dict(negative_slope=0.05),
        pooling_type='avg', pooling_params=dict(kernel_size=3),
    ),
    dict(
        in_size=(3,100,100), out_classes=3,
        channels=[16]*5, pool_every=3, hidden_dims=[100]*1,
        conv_params=dict(kernel_size=2, stride=2, padding=2),
        activation_type='lrelu', activation_params=dict(negative_slope=0.1),
        pooling_type='max', pooling_params=dict(kernel_size=2),
    ),
]

for i, params in enumerate(test_params):
    torch.manual_seed(seed)
    net = CNN(**params)
    print(f"\n=== test {i=} ===")
    print(net)

    torch.manual_seed(seed)
    test_out = net(torch.ones(1, 3, 100, 100))
    print(f'{test_out=}')

    expected_out = torch.load(f'tests/assets/expected_conv_out_{i:02d}.pt')
    print(f'max_diff={torch.max(torch.abs(test_out - expected_out)).item()}')
    test.assertTrue(torch.allclose(test_out, expected_out, atol=1e-3))


=== test i=0 ===
CNN(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (mlp): Sequential(
    (0): Linear(in_features=20000, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=10, bias=True)
  )
)
test_out=tensor([[ 0.0745, -0.1058,  0.0928,  0.0476,  0.0057,  0.0051,  0.0938, -0.0582,
          0.0573,  0.0583]], grad_fn=<AddmmBackward0>)
max_diff=7.45058

/var/folders/tx/l6jnnmt13x50_18kqr2h63080000gn/T/ipykernel_55545/3003510923.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  expected_out = torch.load(f'tests/assets/exp

As before, we'll wrap our model with a `Classifier` that provides the necessary functionality for calculating probability scores and obtaining class label predictions.
This time, we'll use a simple approach that simply selects the class with the highest score.

**TODO**: Implement the `ArgMaxClassifier` in the `hw3/classifier.py` module.

In [11]:
from hw3.classifier import ArgMaxClassifier

model = ArgMaxClassifier(model=CNN(**test_params[0]))

test_image = torch.randint(low=0, high=256, size=(3, 100, 100), dtype=torch.float).unsqueeze(0)
test.assertEqual(model.classify(test_image).shape, (1,))
test.assertEqual(model.predict_proba(test_image).shape, (1, 10))
test.assertAlmostEqual(torch.sum(model.predict_proba(test_image)).item(), 1.0, delta=1e-3)

Let's now load CIFAR-10 to use as our dataset.

In [12]:
data_dir = os.path.expanduser('~/.pytorch-datasets')
ds_train = torchvision.datasets.CIFAR10(root=data_dir, download=True, train=True, transform=tvtf.ToTensor())
ds_test = torchvision.datasets.CIFAR10(root=data_dir, download=True, train=False, transform=tvtf.ToTensor())

print(f'Train: {len(ds_train)} samples')
print(f'Test: {len(ds_test)} samples')

x0,_ = ds_train[0]
in_size = x0.shape
num_classes = 10
print('input image size =', in_size)

Files already downloaded and verified
Files already downloaded and verified
Train: 50000 samples
Test: 10000 samples
input image size = torch.Size([3, 32, 32])


Now as usual, as a sanity test let's make sure we can overfit a tiny dataset with our model. But first we need to adapt our `Trainer` for PyTorch models.

**TODO**:
1. Complete the implementaion of the `ClassifierTrainer` class in the `hw3/training.py` module if you haven't done so already.
2. Set the optimizer hyperparameters in `part4_optim_hp()`, respectively, in `hw3/answers.py`.

In [13]:
from hw3.training import ClassifierTrainer
from hw3.answers import part4_optim_hp

torch.manual_seed(seed)

# Define a tiny part of the CIFAR-10 dataset to overfit it
batch_size = 2
max_batches = 25
dl_train = torch.utils.data.DataLoader(ds_train, batch_size, shuffle=False)

# Create model, loss and optimizer instances
model = ArgMaxClassifier(
    model=CNN(
        in_size, num_classes, channels=[32], pool_every=1, hidden_dims=[100],
        conv_params=dict(kernel_size=3, stride=1, padding=1),
        pooling_params=dict(kernel_size=2),
    )
)

hp_optim = part4_optim_hp()
loss_fn = hp_optim.pop('loss_fn')
optimizer = torch.optim.SGD(params=model.parameters(), **hp_optim)

# Use ClassifierTrainer to run only the training loop a few times.
trainer = ClassifierTrainer(model, loss_fn, optimizer, device)
best_acc = 0
for i in range(25):
    res = trainer.train_epoch(dl_train, max_batches=max_batches, verbose=(i%5==0))
    best_acc = res.accuracy if res.accuracy > best_acc else best_acc
    
# Test overfitting
test.assertGreaterEqual(best_acc, 90)

train_batch:   0%|          | 0/25 [00:00<?, ?it/s]

train_batch:   0%|          | 0/25 [00:00<?, ?it/s]

train_batch:   0%|          | 0/25 [00:00<?, ?it/s]

train_batch:   0%|          | 0/25 [00:00<?, ?it/s]

train_batch:   0%|          | 0/25 [00:00<?, ?it/s]

### Residual Networks

A very common addition to the basic convolutional architecture described above are **shortcut connections**.
First proposed by [He et al. (2016)](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/He_Deep_Residual_Learning_CVPR_2016_paper.pdf), this simple addition has been shown to be a crucial
ingredient in order to achieve effective learning with very deep networks.
Virtually all state of the art image classification models from recent years use this technique.

The idea is to add a shortcut, or skip, around every two or more convolutional layers:

<img src="imgs/resnet_block2.png" width="700" />

**On the left** we see an example of a regular Residual Block, that takes a 64 channel input, and performs two 3X3 convolutions , which are added to the original input.  
**On the right** we see an exapmle of a Bottleneck Residual Block, that takes a 256 channel input, projects it to a 64 channel tensor with a 1X1 convolution, then performs an inner 3X3 convolution, followd by another 1X1 projection convolution back to the original numer of channels, 256. The output is then added to the original input.

Overall, we can denote the structure of the bottleneck channels in the given example as 256->64->64->256, where the first and last arrows denote the 1X1 convolutions, and the middle arrow is the inner convolution.
Note that the 1X1 convolution with the default parameters (in pytorch) is defined such that the only dimension of the tensor that changes is the number of channels.

This adds an easy way for the network to learn identity mappings: set the weight values to be very small.
The outcome is that the convolutional layers learn a **residual** mapping, i.e. some delta that is applied
to the identity map, instead of actually learning a completely new mapping from scratch.


Lets start by implementing a general residual block, representing a structure similar to the above diagrams.
Our residual block will be composed of:
- A "main path" with some number of convolutional layers with ReLU between them. Optionally, we'll also apply dropout and  batch normalization layers (in this order) between the convolutions, before the ReLU.
- A "shortcut path" implementing an identity mapping around the main path. In case of a different number of input/output channels, the shortcut path should contain an additional `1x1` convolution to project the channel dimension.
- The sum of the main and shortcut paths output is passed though a ReLU and returned.

**TODO**: Complete the implementation of the `ResidualBlock`'s `__init__()` method in the `hw3/cnn.py` module.

In [ ]:
from hw3.cnn import ResidualBlock

torch.manual_seed(seed)

resblock = ResidualBlock(
    in_channels=3, channels=[6, 4]*2, kernel_sizes=[3, 5]*2,
    batchnorm=True, dropout=0.2
)
print(resblock)

torch.manual_seed(seed)
test_out = resblock(torch.ones(1, 3, 32, 32))
print(f'{test_out.shape=}')

expected_out = torch.load('tests/assets/expected_resblock_out.pt')
print(f'max_diff={torch.max(torch.abs(test_out - expected_out)).item()}')
test.assertTrue(torch.allclose(test_out, expected_out, atol=1e-3))

ResidualBlock(
  (main_path): Sequential(
    (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout2d(p=0.2, inplace=False)
    (4): Conv2d(6, 4, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout2d(p=0.2, inplace=False)
    (8): Conv2d(4, 4, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (9): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout2d(p=0.2, inplace=False)
    (12): Conv2d(4, 4, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  )
  (shortcut_path): Conv2d(3, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
)
Main Path Output Mean: 0.029574673622846603, Std: 0.25415530800819397
Shortcut Path Output Mean: 0.0989692285656929, Std: 0.5561443567276001
Co

/var/folders/tx/l6jnnmt13x50_18kqr2h63080000gn/T/ipykernel_55545/3959498997.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  expected_out = torch.load('tests/assets/expe

AssertionError: Max diff 1.9073693752288818 exceeds tolerance!

#### Bottleneck Blocks

In the ResNet Block diagram shown above, the right block is called a bottleneck block.
This type of block is mainly used deep in the network, where the feature space becomes increasingly high-dimensional (i.e. there are many channels).

Instead of applying a KxK conv layer on the original input channels, a bottleneck block
first projects to a lower number of features (channels), applies the KxK conv on the result, and then projects back to the original feature space.
Both projections are performed with 1x1 convolutions.

**TODO**: Complete the implementation of the `ResidualBottleneckBlock` in the `hw3/cnn.py` module.

In [44]:
from hw3.cnn import ResidualBottleneckBlock

torch.manual_seed(seed)
resblock_bn = ResidualBottleneckBlock(
    in_out_channels=256, inner_channels=[64, 32, 64], inner_kernel_sizes=[3, 5, 3],
    batchnorm=False, dropout=0.1, activation_type="lrelu"
)
print(resblock_bn)

# Test a forward pass
torch.manual_seed(seed)
test_in  = torch.ones(1, 256, 32, 32)
test_out = resblock_bn(test_in)
print(f'{test_out.shape=}')
assert test_out.shape == test_in.shape 

expected_out = torch.load('tests/assets/expected_resblock_bn_out.pt')
print(f'max_diff={torch.max(torch.abs(test_out - expected_out)).item()}')
test.assertTrue(torch.allclose(test_out, expected_out, atol=1e-3))

ResidualBottleneckBlock(
  (main_path): Sequential(
    (0): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Dropout2d(p=0.1, inplace=False)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.01)
    (5): Dropout2d(p=0.1, inplace=False)
    (6): Conv2d(64, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (7): LeakyReLU(negative_slope=0.01)
    (8): Dropout2d(p=0.1, inplace=False)
    (9): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): LeakyReLU(negative_slope=0.01)
    (11): Dropout2d(p=0.1, inplace=False)
    (12): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
    (13): LeakyReLU(negative_slope=0.01)
    (14): Dropout2d(p=0.1, inplace=False)
    (15): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  )
  (shortcut_path): Identity()
)
Main Path Output Mean: -0.0015658539487048984, Std: 0.04946903511881828
Shortcut Path Output Me

/var/folders/tx/l6jnnmt13x50_18kqr2h63080000gn/T/ipykernel_55545/576432873.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  expected_out = torch.load('tests/assets/expec

AssertionError: False is not true

Now, based on the `ResidualBlock`, we'll implement our own variation of a residual network (ResNet),
with the following architecture:

    [-> (CONV -> ACT)*P -> POOL]*(N/P) -> (FC -> ACT)*M -> FC
     \------- SKIP ------/
     
Note that $N$, $P$ and $M$ are as before, however now $P$ also controls the number of convolutional layers to add a skip-connection to.

**TODO**: Complete the implementation of the `ResNet` class in the `hw3/cnn.py` module.
You must use your `ResidualBlock`s or `ResidualBottleneckBlock`s to group together every $P$ convolutional layers.

In [47]:
from hw3.cnn import ResNet

test_params = [
    dict(
        in_size=(3,100,100), out_classes=10, channels=[32, 64]*3,
        pool_every=4, hidden_dims=[100]*2,
        activation_type='lrelu', activation_params=dict(negative_slope=0.01),
        pooling_type='avg', pooling_params=dict(kernel_size=2),
        batchnorm=True, dropout=0.1,
        bottleneck=False
    ),
    dict(
        # create 64->16->64 bottlenecks
        in_size=(3,100,100), out_classes=5, channels=[64, 16, 64]*4,
        pool_every=3, hidden_dims=[64]*1,
        activation_type='tanh',
        pooling_type='max', pooling_params=dict(kernel_size=2),
        batchnorm=True, dropout=0.1,
        bottleneck=True
    )
]

for i, params in enumerate(test_params):
    torch.manual_seed(seed)
    net = ResNet(**params)
    print(f"\n=== test {i=} ===")
    print(net)

    torch.manual_seed(seed)
    test_out = net(torch.ones(1, 3, 100, 100))
    print(f'{test_out=}')
    
    expected_out = torch.load(f'tests/assets/expected_resnet_out_{i:02d}.pt')
    print(f'max_diff={torch.max(torch.abs(test_out - expected_out)).item()}')
    test.assertTrue(torch.allclose(test_out, expected_out, atol=1e-3))

Main Path Output Mean: 0.00021750903397332877, Std: 0.023202840238809586
Shortcut Path Output Mean: 0.0, Std: 0.0
Combined Output Mean (before ReLU): 0.00021750903397332877, Std: 0.023202840238809586
Final Output Mean (after ReLU): 0.010025519877672195, Std: 0.013569045811891556
Main Path Output Mean: -0.016078632324934006, Std: 0.4018626809120178
Shortcut Path Output Mean: 0.010025519877672195, Std: 0.013569077476859093
Combined Output Mean (before ReLU): -0.0060531143099069595, Std: 0.40195101499557495
Final Output Mean (after ReLU): 0.09943467378616333, Std: 0.25377708673477173

=== test i=0 ===
ResNet(
  (feature_extractor): Sequential(
    (0): ResidualBlock(
      (main_path): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.01)
        (3): Dropout2d(p=0.1, inplace=False)
        (4): Conv2d(32, 64, kernel_s

[autoreload of hw3.cnn failed: Traceback (most recent call last):
  File "/Users/bardiadehbasti/Library/Python/3.12/lib/python/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/bardiadehbasti/Library/Python/3.12/lib/python/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/bardiadehbasti/Library/Python/3.12/lib/python/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/bardiadehbasti/Library/Python/3.12/lib/python/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/bardiadehbasti/Library/Python/3.12/lib/python/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/bardiadehbasti/Library/Python/3.12/lib/python/

RuntimeError: Given groups=1, weight of size [16, 64, 1, 1], expected input[1, 3, 50, 50] to have 64 channels, but got 3 channels instead

## Questions
<a id=part3_4></a>

**TODO** Answer the following questions.

### Question 1

Consider the bottleneck block from the right side of the ResNet diagram above.
Compare it to a regular block that performs a two 3x3 convs directly on the 256-channel input (i.e. as shown in the left side of the diagram, with a different number of channels).
Explain the differences between the regular block and the bottleneck block in terms of:

1. Number of parameters. Calculate the exact numbers for these two examples.
2. Number of floating point operations required to compute an output (qualitative assessment).
3. Ability to combine the input: (1) spatially (within feature maps); (2) across feature maps.


### Question 1: Comparing Regular Block and Bottleneck Block

#### **1. Number of Parameters**

**Regular Block**:
- Each 3x3 convolution in the regular block processes 256 input channels and outputs 256 channels.  
  - First 3x3 conv: \(256 \times 256 \times 3 \times 3 + 256 = 590,080\) parameters.  
  - Second 3x3 conv: Same as above, \(590,080\) parameters.  
  - **Total**: \(590,080 + 590,080 = 1,180,160\) parameters.

**Bottleneck Block**:
- First 1x1 conv reduces dimensions to 64:  
  \(256 \times 64 \times 1 \times 1 + 64 = 16,448\) parameters.  
- Middle 3x3 conv processes 64 channels:  
  \(64 \times 64 \times 3 \times 3 + 64 = 36,928\) parameters.  
- Last 1x1 conv restores dimensions to 256:  
  \(64 \times 256 \times 1 \times 1 + 256 = 16,640\) parameters.  
- **Total**: \(16,448 + 36,928 + 16,640 = 70,016\) parameters.

**Comparison**:
The bottleneck block has significantly fewer parameters (\(70,016\)) compared to the regular block (\(1,180,160\)).

---

#### **2. Number of Floating Point Operations (FLOPs)**

**Regular Block**:
- Each 3x3 convolution involves:  
  \(H \times W \times \text{Input Channels} \times \text{Output Channels} \times 3 \times 3\).  
- For an input size of \(56 \times 56\):  
  - First conv FLOPs: \(56 \times 56 \times 256 \times 256 \times 9 = 18,475,008\).  
  - Second conv FLOPs: Same as above.  
  - **Total**: \(18,475,008 + 18,475,008 = 36,950,016\).

**Bottleneck Block**:
- First 1x1 conv FLOPs:  
  \(56 \times 56 \times 256 \times 64 = 917,504\).  
- Middle 3x3 conv FLOPs:  
  \(56 \times 56 \times 64 \times 64 \times 9 = 1,835,008\).  
- Last 1x1 conv FLOPs:  
  \(56 \times 56 \times 64 \times 256 = 917,504\).  
- **Total**: \(917,504 + 1,835,008 + 917,504 = 3,670,016\).

**Comparison**:
The bottleneck block requires far fewer FLOPs (\(3,670,016\)) compared to the regular block (\(36,950,016\)).

---

#### **3. Ability to Combine Inputs**### Question 1: Comparing Regular Block and Bottleneck Block

#### **1. Number of Parameters**

**Regular Block**:
- Each 3x3 convolution in the regular block processes 256 input channels and outputs 256 channels.  
  - First 3x3 conv: \(256 * 256 * 3 * 3 + 256 = 590,080\) parameters.  
  - Second 3x3 conv: Same as above, \(590,080\) parameters.  
  - **Total**: \(590,080 + 590,080 = 1,180,160\) parameters.

**Bottleneck Block**:
- First 1x1 conv reduces dimensions to 64:  
  \(256 * 64 * 1 * 1 + 64 = 16,448\) parameters.  
- Middle 3x3 conv processes 64 channels:  
  \(64 * 64 * 3 * 3 + 64 = 36,928\) parameters.  
- Last 1x1 conv restores dimensions to 256:  
  \(64 * 256 * 1 * 1 + 256 = 16,640\) parameters.  
- **Total**: \(16,448 + 36,928 + 16,640 = 70,016\) parameters.

**Comparison**:
The bottleneck block has significantly fewer parameters (70,016) compared to the regular block (1,180,160).

---

#### **2. Number of Floating Point Operations (FLOPs)**

**Regular Block**:
- Each 3x3 convolution involves:  
  \(H * W * \text{Input Channels} * \text{Output Channels} * 3 * 3\).  
- For an input size of \(56 * 56\):  
  - First conv FLOPs: \(56 * 56 * 256 * 256 * 9 = 18,475,008\).  
  - Second conv FLOPs: Same as above.  
  - **Total**: \(18,475,008 + 18,475,008 = 36,950,016\).

**Bottleneck Block**:
- First 1x1 conv FLOPs:  
  \(56 * 56 * 256 * 64 = 917,504\).  
- Middle 3x3 conv FLOPs:  
  \(56 * 56 * 64 * 64 * 9 = 1,835,008\).  
- Last 1x1 conv FLOPs:  
  \(56 * 56 * 64 * 256 = 917,504\).  
- **Total**: \(917,504 + 1,835,008 + 917,504 = 3,670,016\).

**Comparison**:
The bottleneck block requires far fewer FLOPs (3,670,016) compared to the regular block (36,950,016).

---

#### **3. Ability to Combine Inputs**

**Regular Block**:
- **Spatial Combination**: Two 3x3 convolutions provide a larger effective receptive field (5 * 5), enhancing spatial learning.  
- **Across Feature Maps**: Processes all 256 channels twice, allowing strong inter-feature map interactions.

**Bottleneck Block**:
- **Spatial Combination**: Only the middle 3x3 convolution contributes to spatial learning, with a smaller receptive field (3 * 3).  
- **Across Feature Maps**: 1x1 convolutions excel at reducing and restoring dimensions, allowing efficient feature map interactions without excessive computations.

---

The bottleneck block is a computationally efficient alternative, trading off some spatial and feature map interactions for reduced parameters and FLOPs, which makes it ideal for deeper networks like ResNet-50 and beyond.

**Regular Block**:
- **Spatial Combination**: Two 3x3 convolutions provide a larger effective receptive field (\(5 \times 5\)), enhancing spatial learning.  
- **Across Feature Maps**: Processes all 256 channels twice, allowing strong inter-feature map interactions.

**Bottleneck Block**:
- **Spatial Combination**: Only the middle 3x3 convolution contributes to spatial learning, with a smaller receptive field (\(3 \times 3\)).  
- **Across Feature Maps**: 1x1 convolutions excel at reducing and restoring dimensions, allowing efficient feature map interactions without excessive computations.

---

#### **Summary Table**

| Metric                         | Regular Block                | Bottleneck Block          |
|--------------------------------|-----------------------------|---------------------------|
| **Parameters**                 | \(1,180,160\)               | \(70,016\)                |
| **FLOPs**                      | \(36,950,016\)              | \(3,670,016\)             |
| **Spatial Combination**        | Strong (\(5 \times 5\))      | Moderate (\(3 \times 3\)) |
| **Feature Map Interaction**    | Extensive                   | Efficient via 1x1 convs   |

The bottleneck block is a computationally efficient alternative, trading off some spatial and feature map interactions for reduced parameters and FLOPs, which makes it ideal for deeper networks like ResNet-50 and beyond.